In [7]:
import time
import gc
import random
import pandas as pd
# import numpy as np
# from tqdm import tqdm
import matplotlib.pyplot as plt


# import xgboost as xgb
# from sklearn.metrics import f1_score
# from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer

random.seed(42)
gc.enable()
%matplotlib inline

# modeling

### Dask_XGB

https://xgboost.readthedocs.io/en/stable/tutorials/dask.html

https://examples.dask.org/machine-learning/text-vectorization.html

https://examples.dask.org/machine-learning/xgboost.html

In [ ]:
import dask_xgboost

import dask.dataframe as dd
from dask.distributed import Client
import dask_ml.feature_extraction.text
from dask_ml.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
params = {'objective': 'multi:softmax',
          'num_class':20}

bst = dask_xgboost.train(client, params, X_train, y_train, num_boost_round=100)

In [ ]:
bst.best_iteration()

In [ ]:
y_hat = dask_xgboost.predict(client, bst, X_test).persist()
y_hat

In [ ]:
y_hat.compute()

In [ ]:
y_test, y_hat = dask.compute(y_test, y_hat)

In [ ]:
score = f1_score(y_test, y_hat, average='macro')

### XGBoost with SKlearn wrapper on Dask

In [5]:
from itertools import repeat

import xgboost as xgb
import dask.array as da
import dask.distributed
import dask.dataframe as dd
import dask_ml.feature_extraction.text

from distributed import LocalCluster, Client

In [9]:
def load_data():
    twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
    
    df = dd.from_pandas(pd.DataFrame({"text": twenty_train.data,
                                      "target": twenty_train.target}),
                        npartitions=25)
    
    vect = dask_ml.feature_extraction.text.HashingVectorizer(lowercase=False, ngram_range=(1, 2))
#     vect = dask_ml.feature_extraction.text.CountVectorizer()
    
    y = df['target'].to_dask_array(lengths=True)
    X = vect.fit_transform(df['text'])
    
    X.compute_chunk_sizes()
    y.compute_chunk_sizes()
    
    return X, y


def main(X, y, client: Client) -> None:
    clf = xgb.dask.DaskXGBClassifier(n_estimators=100, tree_method="hist")
    clf.client = client  # assign the client
    clf.fit(X, y, eval_set=[(X, y)])
#     proba = clf.predict_proba(X)

In [ ]:
if __name__ == "__main__":
    
    num_exp = 2

    all_exps = []
    exp_time = []
    exp_score = []
    experiment_dict ={}
    experiment_dict["method"] = f"Dask workers=4"   

    X, y = load_data()
    
    with LocalCluster() as cluster:
        with Client(cluster) as client:
            for i in repeat(1, num_exp):
                start = time.time()
                main(X, y, client)
                end = time.time()
                exp_time.append(end-start)

    experiment_dict["time_result"] = exp_time
    all_exps.append(experiment_dict)


INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:54166
INFO:distributed.scheduler:  dashboard at:            127.0.0.1:8787
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:54169'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:54170'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:54171'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:54172'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:54177', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:54177
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:54179
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:54180', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:54180
INFO:distributed.core:Starting establi

# Visualize result

In [ ]:
## if you ran the notebook
# df_result = pd.DataFrame(all_exps).set_index("method")
# df_result = df_result.reset_index()

# if you are loading the result dataframe from file
import ast

df_result = pd.read_csv('../output/df_result_joblib_ray_29012023.csv')
df_result['time_result'] = df_result['time_result'].apply(lambda x: ast.literal_eval(x))

In [ ]:
df_result['average_score'][0]

In [ ]:
df_result['time_result_avg'] = df_result['time_result'].apply(lambda x: np.mean(x))

In [ ]:
df_result = df_result.sort_values(by="time_result_avg")

In [ ]:
df_result = df_result.head(2)

In [ ]:
num_exp = 50

fig = plt.figure(figsize=(10, 10))
plt.title(f'XGB performance benchmark for {num_exp} iterations (20 Newsgroup data)')
plt.ylabel("elapsed time (sec)")
fig = df_result["time_result"].head(2).apply(lambda x: pd.Series(x)).T.boxplot(rot=45)
plt.tight_layout()
fig.figure.savefig(f'../img/performance_{num_exp}_joblib_ray.png')


In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.title(f'XGB average time for {num_exp} (20 Newsgroup data)')
plt.ylabel("elapsed time (sec)")

df_result_t = df_result["time_result"].head(2).apply(lambda x: pd.Series(x)).T
fig = df_result_t.reindex(df_result_t.mean().sort_values().index, axis=1).mean().plot(kind='bar', rot=45)

plt.tight_layout()
# fig.figure.savefig(f'../img/average_time_{num_exp}.png')



In [ ]:
# ray https://www.anyscale.com/blog/three-ways-to-speed-up-xgboost-model-training